In [2]:
# Import all required libraries
import pandas as pd
import numpy as np
import pingouin as pg
from pathlib import Path
from scipy import stats
from scipy.stats import spearmanr
import matplotlib.pyplot as plt
import seaborn as sns
import os.path as op
import re
import nibabel as nib

# Additional imports for neuroimaging and atlas processing
from nilearn import datasets, masking, input_data, plotting
from nilearn.maskers import NiftiMapsMasker
from nilearn.connectome import ConnectivityMeasure
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

/var/folders/w7/tr80j6dx7kz1xqvddnxtqtzr0000gn/T/ipykernel_2730/1634084641.py:15: DeprecationWarning: The import path 'nilearn.input_data' is deprecated in version 0.9. Importing from 'nilearn.input_data' will be possible at least until release 0.13.0. Please import from 'nilearn.maskers' instead.
  from nilearn import datasets, masking, input_data, plotting


In [3]:
# Setup directories and plotting theme
RELI_DIR = Path("dset/derivatives/caps/interrater")
FIGURES_DIR = Path("dset/derivatives/figures")
# will use loop later to run over all subjects
OUT_DIR = Path("dset/derivatives/caps/emotion-correlation")
OUT_DIR.mkdir(parents=True, exist_ok=True)

In [6]:
# Path to the unzipped atlas file
craddock_atlas = Path("dset/craddock2012_tcorr05_2level_270_2mm.nii")

crd_nii = nib.load(craddock_atlas)
crd_arr = crd_nii.get_fdata()

print(f"Atlas loaded: {craddock_atlas}")

# Get unique ROI labels (excluding background/0)
unique_labels = np.unique(crd_arr)
print(unique_labels)

Atlas loaded: dset/craddock2012_tcorr05_2level_270_2mm.nii
[  0.   1.   2.   3.   4.   5.   6.   7.   8.   9.  10.  11.  12.  13.
  14.  15.  16.  17.  18.  19.  20.  21.  22.  23.  24.  25.  26.  27.
  28.  29.  30.  31.  32.  33.  34.  35.  36.  37.  38.  39.  40.  41.
  42.  43.  44.  45.  46.  47.  48.  49.  50.  51.  52.  53.  54.  55.
  56.  57.  58.  59.  60.  61.  62.  63.  64.  65.  66.  67.  68.  69.
  70.  71.  72.  73.  74.  75.  76.  77.  78.  79.  80.  81.  82.  83.
  84.  85.  86.  87.  88.  89.  90.  91.  92.  93.  94.  95.  96.  97.
  98.  99. 100. 101. 102. 103. 104. 105. 106. 107. 108. 109. 110. 111.
 112. 113. 114. 115. 116. 117. 118. 119. 120. 121. 122. 123. 124. 125.
 126. 127. 128. 129. 130. 131. 132. 133. 134. 135. 136. 137. 138. 139.
 140. 141. 142. 143. 144. 145. 146. 147. 148. 149. 150. 151. 152. 153.
 154. 155. 156. 157. 158. 159. 160. 161. 162. 163. 164. 165. 166. 167.
 168. 169. 170. 171. 172. 173. 174. 175. 176. 177. 178. 179. 180. 181.
 182. 183. 184. 18

In [7]:
visual = pd.read_csv('/Users/chloehampson/Desktop/diva-emotion-dfc/dset/1visual_crad_rois.csv')
somato = pd.read_csv('/Users/chloehampson/Desktop/diva-emotion-dfc/dset/2somatomotor_crad_rois.csv')
dan = pd.read_csv('/Users/chloehampson/Desktop/diva-emotion-dfc/dset/3dan_crad_rois.csv')
van = pd.read_csv('/Users/chloehampson/Desktop/diva-emotion-dfc/dset/4van_crad_rois.csv')
limbic = pd.read_csv('/Users/chloehampson/Desktop/diva-emotion-dfc/dset/5limbic_crad_rois.csv')
control = pd.read_csv('/Users/chloehampson/Desktop/diva-emotion-dfc/dset/6control_crad_rois.csv')
default = pd.read_csv('/Users/chloehampson/Desktop/diva-emotion-dfc/dset/7default_crad_rois.csv')

crad_labels = pd.Series(index=np.unique(crd_arr), name='Network', dtype=str)

for i in visual['Value'].unique():
    crad_labels.at[i] = 'Visual'
for j in somato['Value'].unique():
    crad_labels.at[j] = 'Somatomotor'
for i in dan['Value'].unique():
    crad_labels.at[i] = 'Dorsal Attention'
for j in van['Value'].unique():
    crad_labels.at[j] = 'Ventral Attention'
for j in limbic['Value'].unique():
    crad_labels.at[j] = 'Limbic'
for i in control['Value'].unique():
    crad_labels.at[i] = 'Control'
for j in default['Value'].unique():
    crad_labels.at[j] = 'Default Mode'

In [8]:
crad_labels.at[1.0] = 'Cerebellum'
crad_labels.at[2.0] = 'Ventral Attention'
crad_labels.at[5.0] = 'Ventral Attention'
crad_labels.at[6.0] = 'Ventral Attention'
crad_labels.at[7.0] = 'Somatomotor'
crad_labels.at[8.0] = 'Cerebellum'
crad_labels.at[9.0] = 'Dorsal Attention'
crad_labels.at[10.0] = 'Somatomotor'
crad_labels.at[13.0] = 'Cerebellum'
crad_labels.at[15.0] = 'Control'
crad_labels.at[18.0] = 'Cerebellum'
crad_labels.at[22.0] = 'Somatomotor'
crad_labels.at[24.0] = 'Control'
crad_labels.at[25.0] = 'Control'
crad_labels.at[27.0] = 'Visual'
crad_labels.at[28.0] = 'Somatomotor'
crad_labels.at[29.0] = 'Ventral Attention'
crad_labels.at[30.0] = 'Limbic'
crad_labels.at[31.0] = 'Control'
crad_labels.at[32.0] = 'Cerebellum'
crad_labels.at[33.0] = 'Somatomotor'
crad_labels.at[34.0] = 'Subcortical'
crad_labels.at[35.0] = 'Cerebellum'
crad_labels.at[36.0] = 'Somatomotor'
crad_labels.at[37.0] = 'Control'
crad_labels.at[38.0] = 'Control'
crad_labels.at[41.0] = 'Ventral Attention'
crad_labels.at[42.0] = 'Visual'
crad_labels.at[43.0] = 'Default Mode'
crad_labels.at[44.0] = 'Visual'
crad_labels.at[45.0] = 'Default Mode'
crad_labels.at[47.0] = 'Dorsal Attention'
crad_labels.at[48.0] = 'Cerebellum'
crad_labels.at[49.0] = 'Visual'
crad_labels.at[50.0] = 'Default Mode'
crad_labels.at[52.0] = 'Subcortical'
crad_labels.at[54.0] = 'Default Mode'
crad_labels.at[56.0] = 'Default Mode'
crad_labels.at[57.0] = 'Ventral Attention'
crad_labels.at[58.0] = 'Subcortical'
crad_labels.at[59.0] = 'Default Mode'
crad_labels.at[60.0] = 'Limbic'
crad_labels.at[61.0] = 'Visual'

crad_labels.at[63.0] = 'Visual'
crad_labels.at[64.0] = 'Somatomotor'
crad_labels.at[65.0] = 'Somatomotor'
crad_labels.at[66.0] = 'Cerebellum'
crad_labels.at[67.0] = 'Cerebellum'

crad_labels.at[68.0] = 'Dorsal Attention'
crad_labels.at[70.0] = 'Default Mode'
crad_labels.at[71.0] = 'Visual'
crad_labels.at[73.0] = 'Default Mode'
crad_labels.at[74.0] = 'Somatomotor'
crad_labels.at[75.0] = 'Ventral Attention'
crad_labels.at[77.0] = 'Limbic'
crad_labels.at[78.0] = 'Default Mode'

crad_labels.at[80.0] = 'Visual'
crad_labels.at[81.0] = 'Cerebellum'
crad_labels.at[82.0] = 'Default Mode'
crad_labels.at[83.0] = 'Dorsal Attention'
crad_labels.at[84.0] = 'Somatomotor'
crad_labels.at[85.0] = 'Default Mode'
crad_labels.at[86.0] = 'Default Mode'
crad_labels.at[87.0] = 'Control'
crad_labels.at[88.0] = 'Visual'
crad_labels.at[89.0] = 'Dorsal Attention'

crad_labels.at[90.0] = ''
crad_labels.at[91.0] = 'Control'
crad_labels.at[92.0] = 'Subcortical'
crad_labels.at[93.0] = 'Ventral Attention'
crad_labels.at[95.0] = 'Ventral Attention'
crad_labels.at[96.0] = 'Limbic'
crad_labels.at[97.0] = 'Visual'
crad_labels.at[98.0] = 'Ventral Attention'

crad_labels.at[101.0] = 'Subcortical'
crad_labels.at[103.0] = 'Ventral Attention'
crad_labels.at[104.0] = 'Default Mode'
crad_labels.at[105.0] = 'Default Mode'
crad_labels.at[106.0] = 'Cerebellum'
crad_labels.at[107.0] = 'Default Mode'

crad_labels.at[108.0] = 'Dorsal Attention'
crad_labels.at[109.0] = 'Subcortical'
crad_labels.at[110.0] = 'Visual'
crad_labels.at[111.0] = 'Somatomotor'
crad_labels.at[112.0] = 'Cerebellum'

crad_labels.at[115.0] = 'Cerebellum'
crad_labels.at[116.0] = 'Somatomotor'

crad_labels.at[119.0] = 'Visual'
crad_labels.at[120.0] = 'Dorsal Attention'
crad_labels.at[121.0] = 'Somatomotor'
crad_labels.at[122.0] = 'Default Mode'

crad_labels.at[124.0] = 'Somatomotor'
crad_labels.at[125.0] = 'Visual'

crad_labels.at[129.0] = 'Default Mode'
crad_labels.at[130.0] = 'Subcortical'
crad_labels.at[131.0] = 'Subcortical'
crad_labels.at[132.0] = 'Visual'

crad_labels.at[133.0] = 'Visual'
crad_labels.at[134.0] = 'Control'
crad_labels.at[135.0] = 'Somatomotor'
crad_labels.at[136.0] = 'Control'

crad_labels.at[137.0] = 'Control'
crad_labels.at[138.0] = 'Visual'
crad_labels.at[139.0] = 'Limbic'
crad_labels.at[140.0] = 'Somatomotor'

crad_labels.at[142.0] = 'Control'
crad_labels.at[144.0] = 'Somatomotor'
crad_labels.at[145.0] = 'Control'
crad_labels.at[143.0] = 'Default Mode'

crad_labels.at[148.0] = 'Control'
crad_labels.at[149.0] = 'Somatomotor'
crad_labels.at[150.0] = 'Default Mode'
crad_labels.at[151.0] = 'Visual'

crad_labels.at[152.0] = 'Cerebellum'
crad_labels.at[153.0] = 'Subcortical'
crad_labels.at[154.0] = 'Default Mode'
crad_labels.at[156.0] = 'Somatomotor'

crad_labels.at[159.0] = 'Control'
crad_labels.at[161.0] = 'Cerebellum'
crad_labels.at[163.0] = 'Default Mode'

crad_labels.at[166.0] = 'Dorsal Attention'
crad_labels.at[167.0] = 'Visual'
crad_labels.at[168.0] = 'Default Mode'
crad_labels.at[169.0] = 'Default Mode'

crad_labels.at[171.0] = 'Subcortical'
crad_labels.at[172.0] = 'Default Mode'
crad_labels.at[173.0] = 'Somatomotor'
crad_labels.at[174.0] = 'Limbic'

crad_labels.at[175.0] = 'Limbic'
crad_labels.at[178.0] = 'Dorsal Attention'
crad_labels.at[180.0] = 'Cerebellum'
crad_labels.at[183.0] = 'Control'
crad_labels.at[186.0] = 'Control'

crad_labels.at[188.0] = 'Cerebellum'
crad_labels.at[189.0] = 'Control'
crad_labels.at[190.0] = 'Default Mode'
crad_labels.at[191.0] = 'Default Mode'
crad_labels.at[192.0] = 'Visual'

crad_labels.at[194.0] = 'Control'
crad_labels.at[195.0] = 'Visual'
crad_labels.at[196.0] = 'Cerebellum'
crad_labels.at[197.0] = 'Dorsal Attention'
crad_labels.at[198.0] = 'Ventral Attention'

crad_labels.at[200.0] = 'Control'
crad_labels.at[201.0] = 'Ventral Attention'
crad_labels.at[203.0] = 'Visual'
crad_labels.at[205.0] = 'Somatomotor'
crad_labels.at[206.0] = 'Dorsal Attention'

crad_labels.at[209.0] = 'Somatomotor'
crad_labels.at[211.0] = 'Somatomotor'
crad_labels.at[212.0] = 'Visual'
crad_labels.at[213.0] = 'Control'

crad_labels.at[221.0] = 'Cerebellum'
crad_labels.at[222.0] = 'Somatomotor'
crad_labels.at[226.0] = 'Visual'

crad_labels.at[232.0] = 'Dorsal Attention'
crad_labels.at[233.0] = 'Subcortical'
crad_labels.at[234.0] = 'Dorsal Attention'

crad_labels.at[236.0] = 'Visual'
crad_labels.at[237.0] = 'Visual'
crad_labels.at[239.0] = 'Cerebellum'

crad_labels.at[250.0] = 'Subcortical'
crad_labels.at[246.0] = 'Subcortical'
crad_labels.at[247.0] = 'Default Mode'
crad_labels.at[249.0] = 'Control'

crad_labels.at[259.0] = 'Visual, Cerebellum'
crad_labels.at[260.0] = 'Cerebellum'

In [9]:
crad_labels[crad_labels.isna()].index#[:10]

Index([], dtype='float64')